# Image patching from LCZ polygons samples on Sen-2 imagery

In [1]:
import csv
import os
import rasterio
from rasterio.windows import Window

### Define image_patch function

In [2]:
def image_patch(image_path, x, y, patch_size, label):
    """
    Extracts a patch of size patch_size x patch_size centered at (x, y) from the image
    located at image_path, and saves it to a new file in a directory named after label.
    """
    # Open the image file
    with rasterio.open(image_path) as src:

        # Compute the window around the datapoint
        row, col = src.index(x, y)
        row_off = row - patch_size // 2
        col_off = col - patch_size // 2
        window = Window(col_off, row_off, patch_size, patch_size)

        # Read the patch from the image
        patch = src.read(window=window)

        # Update the output metadata to reflect the new dimensions
        meta = src.meta.copy()
        meta.update({
            'width': patch_size,
            'height': patch_size,
            'transform': rasterio.windows.transform(window, src.transform)
        })

        # Create the output directory if it doesn't exist
        label_dir = label
        os.makedirs(label_dir, exist_ok=True)

        # Write the patch to a new file
        output_path = os.path.join(label_dir, f'{x}_{y}.tif')
        with rasterio.open(output_path, 'w', **meta) as dst:
            dst.write(patch)

### for image patching  from polygons (if the center point of the patch is within the polygon)

In [9]:
from osgeo import ogr, osr
import numpy as np

# Define the patch size in pixels
patch_size = 32

# Open the image file:
src = rasterio.open('Amsterdam_Sen2.tif')

# Open the polygons shapefile
shapefile = ogr.Open('polygons/Amsterdam_S2_valid.shp')
polygons_layer = shapefile.GetLayer(0)

# Loop over the polygons
for polygon_feature in polygons_layer:

    # Get the label of the polygon
    label = str(polygon_feature.GetField('label'))
    # Get the bounding box of the polygon
    geom = polygon_feature.GetGeometryRef()
    envelope = geom.GetEnvelope()
    min_x, max_x, min_y, max_y = envelope

    # Get the pixel resolution of the image
    x_res, y_res = src.res

    # Compute the number of steps in the x and y directions
    n_x = int(np.ceil((max_x - min_x) / x_res))
    n_y = int(np.ceil((max_y - min_y) / y_res))

    # Define the step size in pixels
    step_size = patch_size // 2

    # Define the x and y coordinates of the grid points in pixels
    x_coords = np.linspace(min_x, max_x - x_res, n_x, dtype=float)[::step_size]
    y_coords = np.linspace(min_y, max_y - y_res, n_y, dtype=float)[::step_size]
    points = [(x, y) for x in x_coords for y in y_coords]
    
    # Loop over the points
    for x, y in points:
        # Check if the point is within the polygon
        point = ogr.Geometry(ogr.wkbPoint)
        point.AddPoint(x, y)
        if geom.Contains(point):
            # Extract the patch and save it to a file
            image_patch('Amsterdam_Sen2.tif', x, y, patch_size, label)
